In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef



from sklearn.model_selection import train_test_split
from sklearn import metrics
from seaborn import heatmap
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt

In [2]:
X=pd.read_csv('final_dataset.csv',header=None)

X=X[4]
X = X.str.split(expand=True)

Y=pd.read_csv('final_dataset.csv',header=None)
Y = Y[5]


In [3]:
X2=pd.read_csv('final_dataset.csv',header=None)
X2=X2[4]
X2

0       -0.02094169 0.10660766 0.057989314 -0.13466349...
1       -0.0011053804 0.062676415 0.061645027 -0.12944...
2       -0.028776443 0.07125651 0.06738785 -0.10789459...
3       -0.0536277 0.10640945 0.06379759 -0.12726858 -...
4       -0.05193024 0.06697094 0.035421222 -0.10692144...
                              ...                        
2036    -0.048396643 0.10845125 0.06963631 -0.15291168...
2037    -0.011545644 0.10413549 0.115345664 -0.2070618...
2038    -0.03951898 0.06703781 0.04597162 -0.09675237 ...
2039    -0.031775676 0.11227458 0.10016729 -0.13883866...
2040    -0.06704124 0.06410579 0.075478345 -0.10943995...
Name: 4, Length: 2041, dtype: object

In [4]:
print(Y.value_counts())


0    1050
1     991
Name: 5, dtype: int64


In [5]:

X = X.astype(float)
X.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
763    float64
764    float64
765    float64
766    float64
767    float64
Length: 768, dtype: object

less values that expected

In [6]:
def cross_val(model, X, Y):
    sensitivity_scorer = make_scorer(recall_score)
    specificity_scorer = make_scorer(recall_score, pos_label=0)


    scoring = {'AUC': 'roc_auc', 'Accuracy': "accuracy", "f1": "f1",
                        "Recall": "recall", "Precision": "precision", "Average Precision": "average_precision",
                        "Sensitivity": sensitivity_scorer, "Specificity": specificity_scorer}

    scores=cross_validate(model, X, Y, scoring=scoring)

    mean_scores = {metric: values.mean() for metric, values in scores.items()}

    mean_scores_df = pd.DataFrame(list(mean_scores.items()), columns=['Metric', 'Mean Score'])

    return mean_scores_df

## Logistic regression

In [15]:
lr=LogisticRegression(C=1)
lr_scores=cross_val(lr,X,Y)
lr_scores

,Metric,Mean Score
0,fit_time,0.147318
1,score_time,0.030222
2,test_AUC,0.711218
3,test_Accuracy,0.663906
4,test_f1,0.636653
5,test_Recall,0.606467
6,test_Precision,0.670577
7,test_Average Precision,0.691599
8,test_Sensitivity,0.606467
9,test_Specificity,0.718095


## LightGBM

In [8]:
lg = LGBMClassifier(verbose=-1,number_estimators=1000,learning_rate=0.1)

lg_scores=cross_val(lg,X,Y)
lg_scores

,Metric,Mean Score
0,fit_time,4.228398
1,score_time,0.038896
2,test_AUC,0.690295
3,test_Accuracy,0.651170
4,test_f1,0.626315
5,test_Recall,0.602442
6,test_Precision,0.653079
7,test_Average Precision,0.684075
8,test_Sensitivity,0.602442
9,test_Specificity,0.697143


## Random forest

In [9]:
rf = RandomForestClassifier()

rf_scores=cross_val(rf,X,Y)
rf_scores

,Metric,Mean Score
0,fit_time,3.018985
1,score_time,0.082179
2,test_AUC,0.689952
3,test_Accuracy,0.650189
4,test_f1,0.616333
5,test_Recall,0.578235
6,test_Precision,0.661134
7,test_Average Precision,0.676713
8,test_Sensitivity,0.578235
9,test_Specificity,0.718095


## SVC

In [10]:
svc = SVC()
svc_scores=cross_val(svc,X,Y)
svc_scores

,Metric,Mean Score
0,fit_time,1.108056
1,score_time,0.775350
2,test_AUC,0.702353
3,test_Accuracy,0.641849
4,test_f1,0.588032
5,test_Recall,0.526714
6,test_Precision,0.666526
7,test_Average Precision,0.689707
8,test_Sensitivity,0.526714
9,test_Specificity,0.750476
